In [8]:
import asyncio
from data_project.gsheets import DateSheet
from datetime import date, timedelta, datetime
import pandas as pd


target_date = (date.today() - timedelta(days=1))

async def create_sheet():
    return DateSheet('[RP_DMM] Revenue Report', date=target_date.isoformat())

task_sheet = asyncio.create_task(create_sheet())
await task_sheet
mysheet = task_sheet.result()

In [9]:
# Change your sheet
mysheet.change_sheet("data", headers_row=2)
sheet = mysheet.worksheet

data = sheet.get_all_values()

# Assuming headers are in the second row (index 1)
headers = data[1]
data = data[2:]

# Create a DataFrame
df = pd.DataFrame(data, columns=headers)

# Filter rows where "Fan Leoi 1" column has the value "Bou Shek"
filtered_df = df[df['分類1'] == '不夜之城']

# Display the filtered DataFrame
print(filtered_df)

ValueError: 'date' is not in list

In [ ]:
# select the target sheet
mysheet = DateSheet('nightless_city_test', date=target_date.isoformat())
mysheet.change_sheet('工作表1')
sheet = mysheet.worksheet
data = filtered_df.copy()
data = data.iloc[: 10]  # only update the first 10 rows

# define the relation of columns
column_names = {
    "userId": "userId",
    "serverPrefix": "serverPrefix",
    "Create Time": "CT",
}

# rename the col of data
data.columns = [column_names.get(c, c).lower() for c in data.columns]

# sort & filter columns
for header in mysheet.headers:
    if header not in data.columns:
        data.loc[:, header] = None
data = data[mysheet.headers]

# sort rows
# data['ct'] = data['ct'].str.replace(" ", "T").apply(datetime.fromisoformat)  # convert str into datetime
data.sort_values("ct", ascending=True, inplace=True)

# compute the cell to paste
cell = f"A{len(sheet.col_values(1)) + 1}"

# update the target sheet
sheet.update(cell, data.values.tolist())